In [1]:
from transformers import GPT2LMHeadModel

In [4]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2")
sd_hf = model_hf.state_dict()

for k,v in sd_hf.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [13]:
import torch
import torch.nn.functional as F

In [15]:
T = 1023
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
tril, wei



(tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 0., 0., 0.],
         ...,
         [1., 1., 1.,  ..., 1., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]]),
 tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [5.0000e-01, 5.0000e-01, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         [3.3333e-01, 3.3333e-01, 3.3333e-01,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00],
         ...,
         [9.7943e-04, 9.7943e-04, 9.7943e-04,  ..., 9.7943e-04, 0.0000e+00,
          0.0000e+00],
         [9.7847e-04, 9.7847e-04, 9.7847e-04,  ..., 9.7847e-04, 9.7847e-04,
          0.0000e+00],
         [9.7752e-04, 9.7752e-04, 9.7752e-04,  ..., 9.7752e-04, 9.7752e-04,
          9.7752e-04]]))

In [18]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2', device='cuda:0')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)

/home/jliu/.conda/envs/llm.c/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the env

[{'generated_text': 'Hello, I\'m a language model, so you can\'t define something in any other language. Let me introduce another topic:\n\nThe name "'},
 {'generated_text': "Hello, I'm a language model, you know.\n\nThat's right… I have a lot of friends who don't know what I do"},
 {'generated_text': "Hello, I'm a language model, not a formal one. I'm more interested in languages than formal models and I'm going to use the formal"},
 {'generated_text': "Hello, I'm a language model, why not add the second one, and also the third one?\n\nA lot of them do not work"},
 {'generated_text': "Hello, I'm a language model, not a dialect. [Laughs.] We've built a database of idioms, a corpus of idioms,"}]

In [21]:
num_return_sequences = 5
max_length = 30

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
model.to('cuda:0')

# prefix token
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
# unsqueeze(dim)：这是 PyTorch 中的一个操作，用于在指定的维度 dim 上增加一个大小为 1 的维度。
# repeat(*sizes)：这是 PyTorch 中的一个操作，用于沿着指定的维度重复张量
x = tokens.to('cuda:0')

torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    with torch.no_grad():
        logits = model(x)[0] # !!! different for huggingface GPT-2 model
        logits = logits[:, -1, :] # logits
        probs = F.softmax(logits, dim=-1)
        # top-k sampling (huggingface pipeline default)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1) # (B, 50)
        ix = torch.multinomial(topk_probs, 1) # (B,1) select from topk probabilities
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1) gather the corresponding indices
        x = torch.cat((x, xcol), dim=1)


# print the generated text
for i in range(num_return_sequences):
    tokens = x[i, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> Hello, I'm a language model, not a program.

So this morning I started studying for the interview in the lab. This was not
> Hello, I'm a language model, and one of the main things that bothers me when they create languages is how easy it becomes to create something that
> Hello, I'm a language model, and I wrote it off on the grounds that a language model would make me more fluent. But I'm not
> Hello, I'm a language model, I really like languages. I like languages because like, they're good. And the way we talk about languages
> Hello, I'm a language model, a language model I'm using for data modelling. All I did was test the results and then I wrote some
